EXtracting and Ingesting Data(Bronz_stage)

In [ ]:
df_payments = spark.read.format("csv") \
    .option("inferSchema", "true") \
    .option("header", "false") \
    .option("sep", ",") \
    .load("dbfs:/FileStore/payments.csv") \
    .toDF("payment_id", "date","amount","rider_id")

display(df_payments.limit(10))

In [ ]:
df_payments.write.format("delta").mode("overwrite").save("/delta/payments")

In [ ]:
df_riders = spark.read.format("csv") \
    .option("inferSchema", "true") \
    .option("header", "false") \
    .option("sep", ",") \
    .load("dbfs:/FileStore/riders.csv") \
    .toDF("rider_id", "first_name", "last_name", "address", "birthday", "account_start_date", "account_end_date", "is_member")

display(df_riders.limit(10))

In [ ]:
df_riders.write.format("delta").mode("overwrite").save("/delta/riders")

In [ ]:
df_stations = spark.read.format("csv") \
    .option("inferSchema", "true") \
    .option("header", "false") \
    .option("sep", ",") \
    .load("dbfs:/FileStore/stations.csv") \
    .toDF("station_id", "name", "latitude", "longitude") 

display(df_stations.limit(10))

In [ ]:
df_stations.write.format("delta").mode("overwrite").save("/delta/stations")

In [ ]:
df_trips = spark.read.format("csv") \
    .option("inferSchema", "true") \
    .option("header", "false") \
    .option("sep", ",") \
    .load("dbfs:/FileStore/trips.csv") \
    .toDF("trip_id", "rideable_type", "start_at", "ended_at", "start_station_id", "end_station_id", "rider_id")

display(df_trips.limit(10))

In [ ]:
df_trips.write.format("delta").mode("overwrite").save("/delta/trips")

CREATE TABLE or SaveAsTables(silver)

In [ ]:
spark.sql("DROP TABLE IF EXISTS stag_payments")
df = spark.read.format("delta").load("/delta/payments")
df.write.format("delta") \
  .mode("overwrite") \
  .saveAsTable("stag_payments")

In [ ]:
spark.sql("DROP TABLE IF EXISTS stag_riders")
spark.sql("CREATE TABLE stag_riders USING DELTA LOCATION '/delta/riders'")

In [ ]:
spark.sql("DROP TABLE IF EXISTS stag_trips")
spark.sql("CREATE TABLE stag_trips USING DELTA LOCATION '/delta/trips'")

In [ ]:
spark.sql("DROP TABLE IF EXISTS stag_stations")
spark.sql("CREATE TABLE stag_stations USING DELTA LOCATION '/delta/stations'")

Transforming Gold

In [ ]:
spark.sql("drop table if exists default.fact_payments")

spark.sql("""
    create table default.fact_payments as 
    select 
        distinct(payment_id), 
        date, 
        amount, 
        rider_id, 
        day(date) AS day, 
        month(date) AS month, 
        quarter(date) AS quarter, 
        year(date) AS year, 
        dayofweek(date) AS day_of_week 
    from default.stag_payments
        """)
     

In [ ]:
spark.sql("drop table if exists default.dim_riders")

spark.sql("""
    create table default.dim_riders as 
    select 
        distinct(rider_id), 
        first_name, 
        last_name, 
        address, 
        birthday, 
        account_start_date, 
        account_end_date, 
        is_member
    from default.stag_riders
        """)
     

In [ ]:
spark.sql("drop table if exists default.dim_stations")

spark.sql("""
    create table default.dim_stations as 
    select 
        distinct(station_id), 
        name, 
        latitude, 
        longitude 
    from default.stag_stations
        """)
     

In [ ]:
spark.sql("drop table if exists curated_data.fact_trips")

spark.sql("""
    create table curated_data.fact_trips as 
    select 
        distinct(trip_id), 
        rideable_type, 
        start_at, 
        ended_at,
        (ended_at - start_at ) as durtion,
        start_station_id,
        end_station_id,
        DATEDIFF(Hour, start_at, ended_at) as duration,        
        stag_trips.rider_id,
        DATEDIFF(YEAR, ri.birthday, start_at) as rider_age
    from default.stag_trips
    left join default.stag_riders as ri on stag_trips.rider_id = ri.rider_id
        """)
     